# Notebook 1 – Data Cleaning, Feature Engineering, & Entity Resolution
**Project:** Judicial Vacancy → Nomination/Confirmation Pipeline

*Initial draft generated via ChatGPT model o3 on 2025-07-12T02:40:38.399372Z*

In [ ]:

import json
import os
import re
import string
import sys
from datetime import datetime
from pathlib import Path

import pandas as pd
from loguru import logger
from rapidfuzz import fuzz, process

# Add the project root to the path so we can import our modules
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

from nomination_predictor.config import INTERIM_DATA_DIR, RAW_DATA_DIR
from nomination_predictor.congress_api_utils import \
    enrich_congress_nominees_dataframe

# Setup logging
logger.remove()  # Remove default handler
logger.add(sys.stderr, format="<green>{time:YYYY-MM-DD HH:mm:ss}</green> | <level>{level}</level> | <cyan>{function}</cyan> - <level>{message}</level>", level="INFO")

2025-07-12 14:27:53.777 | INFO     | nomination_predictor.config:<module>:103 - Project root: /home/wsl2ubuntuuser/nomination_predictor
2025-07-12 14:27:53.779 | INFO     | nomination_predictor.config:<module>:127 - Configuration loaded


5

In [ ]:

# ---------------------------------------------------------------------------
# Helper: clean / normalize names
def clean_name(name: str) -> str:
    if pd.isna(name):
        return ""
    name = str(name).upper()
    name = re.sub(r"[\.,]", "", name)          # drop punctuation
    name = re.sub(r"\s+", " ", name).strip()
    return name

def split_name(name: str):
    """
    Very naive splitter: returns first, middle (maybe empty), last
    """
    parts = clean_name(name).split()
    if not parts:
        return "", "", ""
    if len(parts) == 1:
        return parts[0], "", ""
    if len(parts) == 2:
        return parts[0], "", parts[1]
    return parts[0], " ".join(parts[1:-1]), parts[-1]

def normalised_court(text: str) -> str:
    if pd.isna(text):
        return ""
    text = text.upper().replace("UNITED STATES", "").replace("U.S.", "").strip()
    text = re.sub(r"\s+", " ", text)
    return text

In [ ]:
from nomination_predictor.features import load_and_prepare_dataframes

# Load and prepare all dataframes
dfs = load_and_prepare_dataframes(RAW_DATA_DIR)

# Extract all dataframes with their new names
fjc_judges = dfs["fjc_judges"]
fjc_federal_judicial_service = dfs["fjc_federal_judicial_service"]
fjc_demographics = dfs["fjc_demographics"]
fjc_education = dfs["fjc_education"]
fjc_other_federal_judicial_service = dfs["fjc_other_federal_judicial_service"]
fjc_other_nominations_recess = dfs["fjc_other_nominations_recess"]
seat_timeline = dfs["seat_timeline"]
cong_nominees = dfs["cong_nominees"]
cong_nominations = dfs["cong_nominations"]

print("Loaded:",
      len(dfs["fjc_judges"]), "judges",
      len(dfs["fjc_federal_judicial_service"]), "federal judicial service records",
      len(dfs["fjc_demographics"]), "demographics",
      len(dfs["fjc_education"]), "education",
      len(dfs["fjc_other_federal_judicial_service"]), "other federal judicial service",
      len(dfs["fjc_other_nominations_recess"]), "other nominations recess",
      len(dfs["seat_timeline"]), "seat timeline",
      len(dfs["cong_nominees"]), "congress nominees",
      len(dfs["cong_nominations"]), "nominations",
      )

2025-07-12 14:27:54 | ERROR | load_and_prepare_dataframes - Error loading dataframes: [Errno 2] No such file or directory: '/home/wsl2ubuntuuser/nomination_predictor/data/raw/other_nomination_recess.csv'


FileNotFoundError: [Errno 2] No such file or directory: '/home/wsl2ubuntuuser/nomination_predictor/data/raw/other_nomination_recess.csv'

In [ ]:
print("=== Column Names ===")
print("\nFJC Judges:", fjc_judges.columns.tolist())
print("\nFJC Federal Judicial Service:", fjc_federal_judicial_service.columns.tolist())
print("\nCongress Nominees:", cong_nominees.columns.tolist())
print("\nCongress Nominations:", cong_nominations.columns.tolist())

In [ ]:
# Show basic info about the dataframes
print("=== Basic Info ===")
print("\nFJC Judges shape:", fjc_judges.shape)
print("FJC Federal Judicial Service shape:", fjc_federal_judicial_service.shape)
print("Congress Nominees shape:", cong_nominees.shape)
print("Congress Nominations shape:", cong_nominations.shape)

# Show first few rows of key dataframes
print("\nFirst few FJC Judges:")
display(fjc_judges.head())

print("\nFirst few Congress Nominees:")
display(cong_nominees.head())

Drop non-judge roles from nominations & nominees list

In [ ]:
# Filter out non-judicial nominations using the function from features.py
from nomination_predictor.features import filter_non_judicial_nominations

# Define non-judicial titles to filter out
non_judicial_titles = [
    "Attorney", "Board", "Commission", "Director", "Marshal",
    "Assistant", "Representative", "Secretary of", "Member of"
]

# Apply the filter
cong_nominations, cong_nominees = filter_non_judicial_nominations(
    cong_nominations,
    cong_nominees,
    non_judicial_titles=non_judicial_titles
)

2025-07-12 10:51:23 | INFO | filter_non_judicial_nominations - Found 78 unique citations with non-judicial titles
2025-07-12 10:51:23 | INFO | filter_non_judicial_nominations - Removed 78/285 non-judicial nominations and 78/285 corresponding nominee records


In [ ]:
print("=== Missing Values ===")
print("\nFJC Judges:")
print(fjc_judges.isnull().sum())

print("\nFJC Federal Judicial Service:")
print(fjc_federal_judicial_service.isnull().sum())

print("\nCongress Nominees:")
print(cong_nominees.isnull().sum())

In [ ]:
# --- Clean Congress nominees ------------------------------------------------
cong_nominees["full_name_clean"] = cong_nominees["full_name"].apply(clean_name)
cong_nominees[["first","middle","last"]] = cong_nominees["full_name_clean"].apply(
    lambda n: pd.Series(split_name(n)))

cong_nominees["court_clean"] = cong_nominees["organization"].apply(normalised_court)
cong_nominees["nomination_date"] = pd.to_datetime(cong_nominees["nomination_date"])

# --- Clean FJC judges -------------------------------------------------------
fjc_judges["full_name_clean"] = fjc_judges["name_full"].apply(clean_name)
fjc_judges[["first","middle","last"]] = fjc_judges["full_name_clean"].apply(
    lambda n: pd.Series(split_name(n)))

# We'll need a mapping from nid to service records for date & court validation
fjc_service["court_clean"] = fjc_service["court_name"].apply(normalised_court)
fjc_service["nomination_date"] = pd.to_datetime(fjc_service["nomination_date"], errors="coerce")
fjc_service["commission_date"] = pd.to_datetime(fjc_service["commission_date"], errors="coerce")

In [ ]:

# Block by last name exact match
blocks = {}
for lname, group in fjc_judges.groupby("last"):
    blocks[lname] = group

def candidate_fjc_rows(row):
    return blocks.get(row["last"], pd.DataFrame())

In [ ]:

def best_match(row):
    candidates = candidate_fjc_rows(row)
    if candidates.empty:
        return pd.NA, 0.0
    # Compute combined score: name similarity + court similarity + date proximity
    best_score = 0.0
    best_nid = pd.NA
    for _, cand in candidates.iterrows():
        name_score = fuzz.token_set_ratio(row["full_name_clean"], cand["full_name_clean"])
        # Use service records to find any matching nomination date
        entries = fjc_service[fjc_service["nid"] == cand["nid"]]
        date_score = 0
        court_score = 0
        if not entries.empty:
            # Smallest absolute diff in days
            diffs = (entries["nomination_date"] - row["nomination_date"]).abs().dt.days
            date_score = 100 - diffs.min() if diffs.notna().any() else 0
            # any court string overlap
            if row["court_clean"]:
                if any(row["court_clean"] in c for c in entries["court_clean"]):
                    court_score = 100
                else:
                    court_score = max(fuzz.partial_ratio(row["court_clean"], c) for c in entries["court_clean"])
        total = 0.6*name_score + 0.3*date_score + 0.1*court_score
        if total > best_score:
            best_score, best_nid = total, cand["nid"]
    return best_nid, round(best_score,1)

In [ ]:
# Import the new filter_confirmed_nominees function
from nomination_predictor.features import (analyze_match_failures,
                                           filter_confirmed_nominees,
                                           load_and_prepare_dataframes)

# Load and prepare all dataframes
dfs = load_and_prepare_dataframes(RAW_DATA_DIR)
cong_nominees = dfs["cong_nominees"]  # This now has all the derived fields
fjc_judges = dfs["fjc_judges"]
fjc_service = dfs["fjc_service"]
cong_nominations = dfs["cong_nominations"]

# OPTIMIZATION: Filter to only confirmed nominees before matching
# This saves processing time by only matching nominees who were confirmed
confirmed_nominees = filter_confirmed_nominees(cong_nominees, cong_nominations)
print(f"Focusing on {len(confirmed_nominees)} confirmed nominees out of {len(cong_nominees)} total nominees")

# Only apply best_match to confirmed nominees
confirmed_nominees[["match_nid", "match_score"]] = confirmed_nominees.apply(
    best_match, axis=1, result_type="expand")

# Merge back with original dataframe to preserve all records
# Non-confirmed nominees will have NaN for match fields
cong_nominees = cong_nominees.merge(
    confirmed_nominees[["citation", "match_nid", "match_score"]], 
    on="citation", 
    how="left"
)

In [ ]:

THRESHOLD = 80
matches = cong_nominees[cong_nominees["match_score"] >= THRESHOLD].copy()
print(f"Matched {len(matches)}/{len(cong_nominees)} nominees with score ≥ {THRESHOLD}")
matches.to_csv(INTERIM_DATA_DIR / "congress_fjc_nominee_matches.csv", index=False)

# Save the cleaned interim datasets for downstream notebooks
cong_nominees.to_csv(INTERIM_DATA_DIR / "congress_nominees_cleaned.csv", index=False)
fjc_judges.to_csv(INTERIM_DATA_DIR / "fjc_judges_cleaned.csv", index=False)
fjc_service.to_csv(INTERIM_DATA_DIR / "fjc_service_cleaned.csv", index=False)

Matched 140/207 nominees with score ≥ 80


In [ ]:
from nomination_predictor.features import analyze_match_failures

THRESHOLD = 80
matches = cong_nominees[cong_nominees["match_score"] >= THRESHOLD].copy()
print(f"Matched {len(matches)}/{len(cong_nominees)} nominees with score ≥ {THRESHOLD}")

# Analyze unmatched records to understand why they didn't match
unmatched_df, reason_summary, examples = analyze_match_failures(cong_nominees, THRESHOLD)

# Display summary of failure reasons
print("\nFailure Reason Summary:")
display(reason_summary)

# Display a few examples of each failure type
print("\nExample records for each failure type:")
for reason, example_df in examples.items():
    print(f"\n{reason}:")
    display(example_df)

# Save both matched and unmatched datasets for further analysis
matches.to_csv(INTERIM_DATA_DIR / "congress_fjc_nominee_matches.csv", index=False)
unmatched_df.to_csv(INTERIM_DATA_DIR / "congress_fjc_nominee_unmatched.csv", index=False)

# Save the cleaned interim datasets for downstream notebooks
cong_nominees.to_csv(INTERIM_DATA_DIR / "congress_nominees_cleaned.csv", index=False)
fjc_judges.to_csv(INTERIM_DATA_DIR / "fjc_judges_cleaned.csv", index=False)
fjc_service.to_csv(INTERIM_DATA_DIR / "fjc_service_cleaned.csv", index=False)

Matched 140/207 nominees with score ≥ 80

Failure Reason Summary:


,Failure Reason,Count
0,No potential match candidates found,48
1,Very low similarity - likely different person,3
2,Marginal match (score 76.5) - check name and c...,3
3,Marginal match (score 66.6) - check name and c...,2
4,Marginal match (score 78.5) - check name and c...,2
5,Marginal match (score 51.7) - check name and c...,1
6,Marginal match (score 61.8) - check name and c...,1
7,Marginal match (score 67.4) - check name and c...,1
8,Marginal match (score 77.3) - check name and c...,1
9,Marginal match (score 78.4) - check name and c...,1



Example records for each failure type:

No potential match candidates found:


,full_name,court_clean,match_score,failure_reason
0,James Graham Lake,THE JUDICIARY,0.0,No potential match candidates found
1,Nicholas George Miranda,THE JUDICIARY,0.0,No potential match candidates found
5,Philip S. Hadji,THE JUDICIARY,0.0,No potential match candidates found



Very low similarity - likely different person:


...

,full_name,court_clean,match_score,failure_reason
65,Carmen G. Iguina Gonzalez,THE JUDICIARY,55.1,Marginal match (score 55.1) - check name and c...



Marginal match (score 58.0) - check name and court:


,full_name,court_clean,match_score,failure_reason
153,Charles J. Willoughby Jr.,THE JUDICIARY,58.0,Marginal match (score 58.0) - check name and c...



Marginal match (score 52.4) - check name and court:


,full_name,court_clean,match_score,failure_reason
160,Charles J. Willoughby Jr.,THE JUDICIARY,52.4,Marginal match (score 52.4) - check name and c...


### Build predecessor lookup table

In [ ]:
# Create the predecessor lookup table
predecessor_lookup = get_predecessor_info(seat_timeline_df)
print(f"Created predecessor lookup: {len(predecessor_lookup)} records")

# Preview the predecessor lookup
print(predecessor_lookup.head())
all_dataframes['predecessor_lookup'] = predecessor_lookup